In [26]:
# Import libraries and packages
import pandas as pd
import numpy as np
import os
import math
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings
from functools import reduce
warnings.filterwarnings("ignore")

### Infection subgroup and infection time

In [ ]:
# Load labs - cultures
cultures = pd.read_parquet('/labs/kamaleswaranlab/ECMO/new_data/DR15269_LABsAndPFTs.parquet.gzip')
cultures = cultures[['Pat ID', 'C MRN', 'Encounter CSN', 'Order Date', 'Result Date', 'Procedure', 'Component', 'Result']]
cultures.columns = ['patid', 'mrn', 'csn', 'order_time', 'result_time', 'procedure', 'component', 'result']
cultures[['order_time', 'result_time']] = cultures[['order_time', 'result_time']].apply(pd.to_datetime)
cultures['csn'] = cultures['csn'].astype(int)
cultures = cultures[cultures['procedure'].str.contains('culture', case=False, na=False)]

# Add hosp admission to cultures
dept = pd.read_parquet('/labs/kamaleswaranlab/ECMO/new_data/TAB2_Encounter_Departments.parquet.gzip')
dept = dept[['Encounter CSN', 'Hosp_Admission']]
dept.columns = ['csn', 'hosp_admission']
dept['csn'] = dept['csn'].astype(int)
dept['hosp_admission'] = dept['hosp_admission'].apply(pd.to_datetime)
dept.drop_duplicates(inplace=True)
cultures = cultures.merge(dept, how='left', on='csn')

# Gather data within first 7 days of the stay
cultures['rel_day_cult'] = np.ceil((cultures['order_time'] - cultures['hosp_admission']) / pd.Timedelta('1 day'))
cultures = cultures[cultures['rel_day_cult'] <= 7]
cultures = cultures[['csn', 'order_time', 'rel_day_cult']]
cultures.drop_duplicates(inplace=True)

# Load antiinfective meds
antiinf = pd.read_parquet('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/antiinf_meds.parquet.gzip')
antiinf['mar_time'] = antiinf['mar_time'].apply(pd.to_datetime)
antiinf['csn'] = antiinf['csn'].astype(int)

# Add hosp admission to antiinf
antiinf = antiinf.merge(dept, how='left', on='csn')

# Gather data within first 7 days of the stay
antiinf['rel_day_ant'] = np.ceil((antiinf['mar_time'] - antiinf['hosp_admission']) / pd.Timedelta('1 day'))
antiinf = antiinf[antiinf['rel_day_ant'] <= 7]

# Add culture order time to antiinf
antiinf = antiinf.merge(cultures, how='left', on='csn')

# Gather patients with at least 1 administration of antibiotics during the first day of admission or
# one dose and a culture order within the same 24 hour window 
antiinf = antiinf[(antiinf['rel_day_ant'] == 1) | (antiinf['rel_day_ant'] == antiinf['rel_day_cult'])]
antiinf['inf_time'] = antiinf[['mar_time','order_time']].min(axis=1)
antiinf = antiinf[['csn', 'rel_day_ant', 'inf_time']]
antiinf.columns = ['csn', 'rel_day_inf', 'inf_time']
antiinf.drop_duplicates(inplace=True)
antiinf = antiinf.sort_values(by=['csn', 'rel_day_inf', 'inf_time']).groupby(['csn', 'rel_day_inf'], as_index=False).first()
print(antiinf['csn'].unique().shape[0])

### Phoenix

In [ ]:
# Load data
data = pd.read_pickle('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/variables.pkl')
data = data[data['variable_name'].isin(['pao2_fio2', 'pao2', 'fio2', 'spo2', 'resp_indicator', 'mv_indicator', 'lactic_acid' , 'map', 'resp', 'o2_flow', 'weight',
                                        'platelets', 'inr', 'ddimer', 'fibrinogen', 'coma_scale', 'pupil_right_reaction', 'pupil_left_reaction'])]
data[['dob', 'recorded_time']] = data[['dob', 'recorded_time']].apply(pd.to_datetime)
data['csn'] = data['csn'].astype(int)
variables = data['variable_name'].unique().tolist()

# Load meds
meds = pd.read_parquet('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/filtered_meds.parquet.gzip')
meds[['dob', 'mar_time']] = meds[['dob', 'mar_time']].apply(pd.to_datetime)
meds['csn'] = meds['csn'].astype(int)
meds = meds[(meds['csn'].isin(data['csn'].unique().tolist())) & (meds['dose_unit'] == 'mcg/kg/min') & 
        (meds['med'].str.contains('epinephrine|dopamine|dobutamine|milrinone|vasopressin', case=False))]
meds = meds[['patid', 'csn', 'dob', 'med_id', 'med', 'mar_time', 'dose']]
meds.columns = ['patid', 'csn', 'dob', 'variable_id', 'variable_name', 'recorded_time', 'value']
meds.loc[(meds['variable_name'].str.contains("epinephrine", case=False)) & ~(meds['variable_name'].str.contains("norepinephrine", case=False)), 'variable_name'] = 'epinephrine'
meds.loc[meds['variable_name'].str.contains("norepinephrine", case=False), 'variable_name'] = 'norepinephrine'
meds.loc[meds['variable_name'].str.contains("dopamine", case=False), 'variable_name'] = 'dopamine'
meds.loc[meds['variable_name'].str.contains("dobutamine", case=False), 'variable_name'] = 'dobutamine'
meds.loc[meds['variable_name'].str.contains("milrinone", case=False), 'variable_name'] = 'milrinone'
meds.loc[meds['variable_name'].str.contains("vasopressin", case=False), 'variable_name'] = 'vasopressin'
meds.reset_index(inplace=True, drop=True)
data = pd.concat([data, meds])

# Organize pupillary reaction
data.loc[(data['variable_name'].isin(['pupil_right_reaction', 'pupil_left_reaction'])) & (data['value'] == 'Non-reactive'), 'value'] = 0
data.loc[(data['variable_name'].isin(['pupil_right_reaction', 'pupil_left_reaction'])) & (data['value'] == 'Reactive'), 'value'] = 1
data.loc[(data['variable_name'].isin(['pupil_right_reaction', 'pupil_left_reaction'])) & (data['value'] == 'Unable to Assess'), 'value'] = 2

# Discard NaN
data.dropna(subset='value', inplace=True)

# Remove invalid values
data = data[data['value'].apply(lambda x: str(x).replace(".", "", 1).isdigit())]
data['value'] = data['value'].astype(float)
data.reset_index(inplace=True, drop=True)
data = data[~((data['variable_name'] == 'spo2') & (data['value'] > 97))]

# Convert weight from oz to lb
data.loc[data['variable_name'] == 'weight', 'value'] = data.loc[data['variable_name'] == 'weight', 'value'].apply(lambda x: round(x/16 ,2))

# Load departments data
dept = pd.read_parquet('/labs/kamaleswaranlab/ECMO/new_data/TAB2_Encounter_Departments.parquet.gzip')
dept = dept[['Encounter CSN', 'MRN', 'Hosp_Admission']]
dept.drop_duplicates(inplace=True)
dept.columns = ['csn', 'mrn', 'hosp_admission']
dept['csn'] = dept['csn'].astype(int)
dept[['hosp_admission']] = dept[['hosp_admission']].apply(pd.to_datetime)

# Add hospital admission and mrn
data = data.merge(dept, how='inner', on='csn')
data.drop_duplicates(inplace=True)
data.reset_index(drop=True, inplace=True)
data = data[['patid', 'mrn', 'csn', 'dob', 'hosp_admission', 'variable_id', 'variable_name', 'recorded_time', 'value']]

# Compute age in months and years
data['age_days'] = round((data['hosp_admission'] - data['dob']) / pd.Timedelta('1 day'), 0)
data['age_months'] = round(data['age_days'] / 31, 2)
data['age_years'] = round(data['age_days'] / 365.25, 2)
data.drop('age_days', axis=1, inplace=True)
data = data[['patid', 'mrn', 'csn', 'dob', 'age_months', 'age_years', 'hosp_admission', 'variable_id', 'variable_name', 'recorded_time', 'value']]

# Gather data within the first 7 days of the stay
data['rel_day'] = np.ceil((data['recorded_time'] - data['hosp_admission']) / pd.Timedelta('1 day'))
data = data[data['rel_day'] <= 7]
data.drop('rel_day', axis=1, inplace=True)

# Load demographics file
demo = pd.read_parquet('/labs/kamaleswaranlab/ECMO/new_data/TAB1_Patients.parquet.gzip')
demo = demo[['Pat ID', 'Gender']]
demo.columns = ['patid', 'gender']
demo.drop_duplicates(inplace=True)

# Add gender
data = data.merge(demo, how='inner', on='patid')
data.drop_duplicates(inplace=True)
data.reset_index(drop=True, inplace=True)
data = data[['patid', 'mrn', 'csn', 'dob', 'gender', 'age_months', 'age_years', 'hosp_admission', 'variable_id', 'variable_name', 'recorded_time', 'value']]

# Pivot dataframe
variables = data['variable_name'].unique().tolist()
data.drop(['variable_id'], axis=1, inplace=True)
data = pd.pivot_table(data, values='value', index=['patid', 'mrn', 'csn', 'dob', 'gender', 'age_months', 'age_years', 'hosp_admission', 'recorded_time'], columns='variable_name', aggfunc='median', fill_value=np.nan)
data.reset_index(inplace=True)
data[['dob', 'hosp_admission', 'recorded_time']] = data[['dob', 'hosp_admission', 'recorded_time']].apply(pd.to_datetime)

# Remove outliers
variables.remove('pupil_right_reaction')
variables.remove('pupil_left_reaction')
variables.remove('resp_indicator')
variables.remove('mv_indicator')
for var in variables:
    p1 = np.nanpercentile(data[var], 1.0)
    p99 = np.nanpercentile(data[var], 99.0)
    data.loc[data[var] < p1, var] = np.nan
    data.loc[data[var] > p99, var] = np.nan

# Calculate PaO2/FiO2
data.loc[data['fio2'] < 21, 'fio2'] = 21
data['fio2_imputed'] = data.groupby(['csn'])['fio2'].ffill()
data['pao2_fio2_calculated'] = data['pao2'] / (data['fio2_imputed'] / 100)

# Calculate SpO2/FiO2
data['spo2_fio2_calculated'] = data['spo2'] / (data['fio2_imputed'] / 100)
data.drop(['fio2_imputed'], axis=1, inplace=True)

# Gather patients with infection
print('Number of CSNs:', len(data['csn'].unique().tolist()))
data = data[data['csn'].isin(antiinf['csn'].unique().tolist())]
print('Number of CSNs with infection:', len(data['csn'].unique().tolist()))
data.sort_values(by=['csn', 'recorded_time'], inplace=True)
data.reset_index(drop=True, inplace=True)

susp_inf = data[['patid', 'mrn', 'csn', 'dob', 'hosp_admission']]
susp_inf.drop_duplicates(inplace=True)
susp_inf.to_csv('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/susp_inf_phoenix.csv', index=False)

In [ ]:
# Create column with relative time
data['rel_time'] = np.ceil((data['recorded_time'] - data['hosp_admission']) / pd.Timedelta('30 minutes'))
data = data[data['rel_time'] > 0]
data.sort_values(by=['csn', 'rel_time'], inplace=True)
print('Unique CSN total:', len(data['csn'].unique().tolist()))

# Resample data
agg_dict = {}
for col in data.columns:
    if col in ['pao2_fio2_calculated', 'spo2_fio2_calculated', 'map', 'platelets' 'fibrinogen', 'coma_scale', 'pupil_right_reaction', 'pupil_left_reaction']:
        agg_dict[col] = pd.NamedAgg(column=col, aggfunc='min')
    elif col in ['lactic_acid', 'inr', 'ddimer']:
        agg_dict[col] = pd.NamedAgg(column=col, aggfunc='max')
    else:
        agg_dict[col] = pd.NamedAgg(column=col, aggfunc='last')

data = data.groupby(['patid', 'mrn', 'csn', 'dob', 'gender', 'age_months', 'age_years', 'hosp_admission', 'rel_time'], as_index=False).agg(**agg_dict)
data.sort_values(by=['csn', 'rel_time'], inplace=True)
data.reset_index(drop=True, inplace=True)
print('Unique CSN total:', len(data['csn'].unique().tolist()))

# Create rows for missing rows
hours_list = []
csn_list = []
data['rel_time'] = data['rel_time'].astype(int)

for csn in data['csn'].unique().tolist():
    df = data[data['csn'] == csn]
    hours = [x for x in list(range(df['rel_time'].min(), df['rel_time'].max())) if x not in list(df['rel_time'])]
    csn_list.extend([csn] * len(hours))
    hours_list.extend(hours)
missing = pd.DataFrame(list(zip(csn_list, hours_list)), columns=['csn', 'rel_time'])

cols = list(data.columns)
cols.remove('csn')
cols.remove('rel_time')

for col in cols:
    missing[col] = np.nan
    
missing = missing[list(data.columns)]
data = pd.concat([data, missing])
data.sort_values(by=['csn', 'rel_time'], inplace=True)
data.reset_index(inplace=True, drop=True)
data[['patid', 'mrn', 'dob', 'gender', 'age_months', 'age_years', 'hosp_admission']] = data.groupby('csn')[['patid', 'mrn', 'dob', 'gender', 'age_months', 'age_years', 'hosp_admission']].ffill()
data['rel_time_med'] = (data['rel_time'] - 0.5) / 2
data.loc[data['recorded_time'].isna(), 'recorded_time'] = data['hosp_admission'] + pd.to_timedelta(data['rel_time_med'], unit='h')
data.drop(['rel_time_med'], axis=1, inplace=True)
data['rel_day'] = np.ceil((data['recorded_time'] - data['hosp_admission']) / pd.Timedelta('1 day'))
data = data[data['rel_day'] <= 7]
print('Unique CSN total:', len(data['csn'].unique().tolist()))

In [ ]:
# Forward fill 
ff_48 = ['platelets', 'inr', 'fibrinogen', 'ddimer']
ff_24 = ['coma_scale', 'epinephrine', 'norepinephrine', 'dopamine', 'dobutamine', 'milrinone']
ff_12 = ['pao2_fio2_calculated', 'spo2_fio2_calculated', 'resp_indicator', 'mv_indicator', 'map', 'lactic_acid', 'pupil_left_reaction', 'pupil_right_reaction']
data[ff_48] = data.groupby(['patid', 'mrn', 'csn', 'dob', 'gender', 'age_months', 'age_years', 'hosp_admission'])[ff_48].ffill(limit=48)
data[ff_24] = data.groupby(['patid', 'mrn', 'csn', 'dob', 'gender', 'age_months', 'age_years', 'hosp_admission'])[ff_24].ffill(limit=24)
data[ff_12] = data.groupby(['patid', 'mrn', 'csn', 'dob', 'gender', 'age_months', 'age_years', 'hosp_admission'])[ff_12].ffill(limit=12)
print('Unique CSN total:', len(data['csn'].unique().tolist()))

### Phoenix components

In [33]:
# Calculate component scores

# Calculate respiratory component
data['resp_score'] = 0
data['resp_score'] = np.where((data['pao2_fio2_calculated'] >= 400) | (data['spo2_fio2_calculated'] >= 292), 0, 
                        np.where(((data['pao2_fio2_calculated'] < 400) | (data['spo2_fio2_calculated'] < 292)) & (data['resp_indicator'] == 1), 1, 
                        np.where((((data['pao2_fio2_calculated'] >= 100) & (data['pao2_fio2_calculated'] < 200)) | 
                                 ((data['spo2_fio2_calculated'] >= 148) & (data['spo2_fio2_calculated'] < 220))) & (data['mv_indicator'] == 1), 2, 
                        np.where(((data['pao2_fio2_calculated'] < 100) | (data['spo2_fio2_calculated'] < 148)) & (data['mv_indicator'] == 1), 3, 0))))


# Calculate cardiovascular component

data['card_score_1_lactate'] = 0
data.loc[(data['lactic_acid'] >= 5) & (data['lactic_acid'] < 11), 'card_score_1_lactate'] = 1

data['card_score_1_map'] = 0
data.loc[((data['age_months'] < 1) & ((data['map'] >= 17) & (data['map'] <= 30))) | 
        (((data['age_months'] >= 1) & (data['age_years'] < 1)) & ((data['map'] >= 25) & (data['map'] <= 38))) |
        (((data['age_years'] >= 1) & (data['age_years'] < 2)) & ((data['map'] >= 31) & (data['map'] <= 43))) |
        (((data['age_years'] >= 2) & (data['age_years'] < 5)) & ((data['map'] >= 32) & (data['map'] <= 44))) |
        (((data['age_years'] >= 5) & (data['age_years'] < 12)) & ((data['map'] >= 36) & (data['map'] <= 48))) |
        (((data['age_years'] >= 12) & (data['age_years'] < 17)) & ((data['map'] >= 38) & (data['map'] <= 51))), 'card_score_1_map'] = 1

data['card_score_epinephrine'] = 0
data.loc[~(data['epinephrine'].isna()), 'card_score_epinephrine'] = 1
data['card_score_norepinephrine'] = 0
data.loc[~(data['norepinephrine'].isna()), 'card_score_norepinephrine'] = 1
data['card_score_dopamine'] = 0
data.loc[~(data['dopamine'].isna()), 'card_score_dopamine'] = 1
data['card_score_dobutamine'] = 0
data.loc[~(data['dobutamine'].isna()), 'card_score_dobutamine'] = 1
data['card_score_milrinone'] = 0
data.loc[~(data['milrinone'].isna()), 'card_score_milrinone'] = 1

data['card_score_2_lactate'] = 0
data.loc[data['lactic_acid'] >= 11, 'card_score_2_lactate'] = 2

data['card_score_2_map'] = 0
data.loc[((data['age_months'] < 1) & (data['map'] <= 30)) | 
        (((data['age_months'] >= 1) & (data['age_years'] < 1)) & (data['map'] < 17)) |
        (((data['age_years'] >= 1) & (data['age_years'] < 2)) & (data['map'] < 25)) |
        (((data['age_years'] >= 2) & (data['age_years'] < 5)) & (data['map'] < 31)) |
        (((data['age_years'] >= 5) & (data['age_years'] < 12)) & (data['map'] < 32)) |
        (((data['age_years'] >= 12) & (data['age_years'] < 17)) & (data['map'] < 38)), 'card_score_2_map'] = 2


# Calculate coagulation component
data['coag_score_platelets'] = 0
data.loc[data['platelets'] < 100, 'coag_score_platelets'] = 1

data['coag_score_inr'] = 0
data.loc[data['inr'] > 1.3, 'coag_score_inr'] = 1

data['coag_score_ddimer'] = 0
data.loc[data['ddimer'] > 1000, 'coag_score_ddimer'] = 1

data['coag_score_fibrinogen'] = 0
data.loc[data['fibrinogen'] < 100, 'coag_score_fibrinogen'] = 1


# Calculate neurologic component 
data['neuro_score'] = 0
data['neuro_score'] = np.where((data['coma_scale'] > 10) & (data['pupil_left_reaction'] == 1) & (data['pupil_right_reaction'] == 1), 0, 
                        np.where(data['coma_scale'] <= 10, 1, 
                        np.where((data['pupil_left_reaction'] == 0) & (data['pupil_right_reaction'] == 0), 2, 0)))

In [ ]:
# Calculate Phoenix score per rel_time

# data = data[['patid', 'mrn', 'csn', 'dob', 'hosp_admission', 'rel_day', 'resp_score', 'card_score_1_lactate', 
#              'card_score_1_map', 'card_score_epinephrine', 'card_score_norepinephrine', 'card_score_dopamine', 
#              'card_score_dobutamine', 'card_score_milrinone', 'card_score_2_lactate', 'card_score_2_map', 'coag_score_platelets', 
#              'coag_score_inr', 'coag_score_ddimer', 'coag_score_fibrinogen', 'neuro_score']]

# Compute cardiovascular score per rel_time
data['vasoactive_meds'] = data['card_score_epinephrine'] + data['card_score_norepinephrine'] + data['card_score_dopamine'] + data['card_score_dobutamine'] + data['card_score_milrinone']
data['vasoactive_meds'] = np.where(data['vasoactive_meds'] >= 2, 2, np.where(data['vasoactive_meds'] == 1, 1, 0))
data['card_score'] = data['vasoactive_meds'] + data['card_score_1_lactate'] + data['card_score_1_map'] + data['card_score_2_lactate'] + data['card_score_2_map']

# Compute coagulation final score
data['coag_score'] = data['coag_score_platelets'] + data['coag_score_inr'] + data['coag_score_ddimer'] + data['coag_score_fibrinogen']
data.loc[data['coag_score'] > 2, 'coag_score'] = 2

# Compute Phoenix score per rel_time
data['phoenix'] = data['resp_score'] + data['card_score'] + data['coag_score'] + data['neuro_score']

# Select Phoenix score >= 2
data = data[data['phoenix'] >= 2]
print('Unique CSN total:', len(data['csn'].unique().tolist()))
data = data.sort_values(by=['csn', 'recorded_time']).groupby(['csn', 'rel_day'], as_index=False).nth(0)
print('Unique CSN total:', len(data['csn'].unique().tolist()))

In [36]:
# Add infection time
data = data.merge(antiinf, how='inner', on='csn')

# Find patients with sepsis
data = data[data['rel_day'] == data['rel_day_inf']]
data = data.sort_values(by=['csn', 'recorded_time']).groupby('csn', as_index=False).nth(0)
data['sepsis'] = 1

# Add sepsis_time
data['sepsis_time'] = np.nan
data.loc[data['sepsis'] == 1, 'sepsis_time'] = data['hosp_admission'] + pd.to_timedelta(data['rel_day'], unit='d')

# Save file
data.to_pickle('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/screening_inf_phoenix.pkl')

In [ ]:
# Save reduced version
data = data[['patid', 'mrn', 'csn', 'dob', 'inf_time', 'sepsis', 'sepsis_time']]
data.drop_duplicates(inplace=True)
data.reset_index(drop=True, inplace=True)
print('Number of suspected infection CSNs with sepsis: {}.'.format(len(data.loc[data['sepsis'] == 1, 'csn'].unique().tolist())))

In [39]:
# Save positive cohort
data_pos = data[~(data['sepsis_time'].isna())]
data_pos.to_csv('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/cohort_inf_phoenix.csv', index=False)